In [2]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
file_wheat = "/Volumes/toshiba/bio/IWGSC/by_chr/iwgsc_refseqv1.0_chr3A.fsa"

In [3]:
file_genes = "/Volumes/toshiba/bio/daniela/counts/svevo.counts.fa"
file_limit = "/Volumes/toshiba/bio/daniela/counts/svevo.limit.fa"
gene_list = ['TRITD3Av1G026860','TRITD3Av1G027040','TRITD3Av1G036950','TRITD3Av1G036910']

In [10]:
file_genes = "/Volumes/toshiba/bio/daniela/counts/zav_v2.counts.fa"
file_limit = "/Volumes/toshiba/bio/daniela/counts/zavitan.limit.fa"
gene_list = ['TRIDC3Av2G023690','TRIDC3Av2G024160','TRIDC3Av2G034660', 'TRIDC3Av2G034210', 'TRIDC3Av2G034600','TRIDC3Av2G034360']


In [11]:
fasta_file = SeqIO.parse(file_genes, 'fasta')
buffer_seqs = []
for record in fasta_file:
    if record.id in gene_list:
        buffer_seqs.append(record)
SeqIO.write(buffer_seqs, file_limit, "fasta")


6

In [ ]:
# cat disk/counts/svevo.limit.fa disk/counts/zavitan.limit.fa > disk/counts/limits.fa
# blastn -perc_identity 85  -query /media/juan/toshiba/bio/daniela/counts/limits.fa -subject /media/juan/toshiba/bio/IWGSC/by_chr/iwgsc_refseqv1.0_chr3A.fsa  -outfmt "6 qseqid sseqid pident mismatch gapopen qstart qend sstart send evalue length qlen slen"  -evalue 10e-10  > /media/juan/toshiba/bio/daniela/counts/wheat_limits.csv


In [16]:
file_blast = 'disk/counts/wheat_limits.csv'
df_blast = pd.read_csv(file_blast, sep='\t', header=None)
df_blast.columns = ['qseqid','sseqid','pident','mismatch','gapopen','qstart','qend','sstart','send','evalue','length','qlen','slen']
df_blast.head(3)

,qseqid,sseqid,pident,mismatch,gapopen,qstart,qend,sstart,send,evalue,length,qlen,slen
0,TRITD3Av1G026860,chr3A,99.541,14,1,1,3269,58683534,58686801,0.000000e+00,3269,3269,750843639
1,TRITD3Av1G026860,chr3A,96.996,7,0,129,361,58071300,58071532,1.580000e-106,233,3269,750843639
2,TRITD3Av1G026860,chr3A,89.908,11,0,1171,1279,585500962,585501070,6.320000e-31,109,3269,750843639


In [22]:
df_blast = df_blast[((df_blast.length / df_blast.qlen) >= 0.7) &
       ((df_blast.length / abs(df_blast.sstart - df_blast.send)) >= 0.7) &
       ((df_blast.length / df_blast.qlen) <= 1.3) &
       ((df_blast.length / abs(df_blast.sstart - df_blast.send)) <= 1.3)]
df_blast.head(3)

,qseqid,sseqid,pident,mismatch,gapopen,qstart,qend,sstart,send,evalue,length,qlen,slen
0,TRITD3Av1G026860,chr3A,99.541,14,1,1,3269,58683534,58686801,0.0,3269,3269,750843639
192,TRITD3Av1G027040,chr3A,99.810,2,2,1162,4852,58927288,58930973,0.0,3691,4852,750843639
239,TRITD3Av1G036910,chr3A,100.000,0,0,1,2060,83720644,83722703,0.0,2060,2060,750843639


In [23]:
mini = min(min(df_blast.send),min(df_blast.sstart))
maxi = max(max(df_blast.send),max(df_blast.sstart))
mini -= 100000
maxi += 100000
mini, maxi

In [24]:
fasta_file = SeqIO.parse(file_wheat, 'fasta')
buffer_seqs = []
for record in fasta_file:
    if record.id in gene_list:
        buffer_seqs.append(record)
SeqIO.write(buffer_seqs, file_limit, "fasta")


(58683534, 83753545)